In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pip-install-lifelines/tzdata-2024.2-py2.py3-none-any.whl
/kaggle/input/pip-install-lifelines/numpy-2.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/pip-install-lifelines/kiwisolver-1.4.7-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
/kaggle/input/pip-install-lifelines/matplotlib-3.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
/kaggle/input/pip-install-lifelines/scipy-1.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/pip-install-lifelines/pillow-11.0.0-cp310-cp310-manylinux_2_28_x86_64.whl
/kaggle/input/pip-install-lifelines/contourpy-1.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/pip-install-lifelines/pyparsing-3.2.0-py3-none-any.whl
/kaggle/input/pip-install-lifelines/wrapt-1.17.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.1.1-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl

Processing /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... - done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=06324a5bd0a29761a29e0a901e39ecf435e27d54cdf5289701d42a27a04b3cfb
  Stored in directory: /root/.cache/pip/wheels/6b/b5/e0/4c79e15c0b5f2c15ecf613c720bb20daab20a666eb67135155
Successfully built autograd-gamma
Processing /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/formulaic-1.1.1-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl


In [3]:
import pandas as pd
import numpy as np

train = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
test = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')

efs = train['efs'].copy()
efs_time = train['efs_time'].copy()
columns = [col for col in train.columns if col not in ['efs','efs_time']]
print(columns)

['ID', 'dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia', 'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status', 'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6', 'hla_match_c_low', 'rituximab', 'hla_match_drb1_low', 'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe', 'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer', 'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue', 'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score', 'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose', 'hla_low_res_8', 'cardiac', 'hla_match_drb1_high', 'pulm_moderate', 'hla_low_res_10']


In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

unique_values_per_column = train.apply(lambda x: x.unique())
print(unique_values_per_column)

binary_cols = ['psych_disturb','diabetes','arrhythmia','vent_hist','renal_issue',
              'pulm_severe','rituximab','obesity','mrd_hct','in_vivo_tcd',
               'hepatic_severe','prior_tumor','peptic_ulcer','rheum_issue',
               'hepatic_mild','cardiac','pulm_moderate']



ordinal_numeric_cols = ['karnofsky_score','comorbidity_score'] + [col for col in train.columns if 'hla' in col]

ordinal_cols = ['dri_score','cyto_score','cyto_score_detail']

nominal_cols = ['tbi_status','graft_type','prim_disease_hct','cmv_status',
               'tce_imm_match','prod_type','conditioning_intensity',
                'ethnicity','tce_match','gvhd_proph','sex_match','race_group',
               'tce_div_match','melphalan_dose','donor_related']
numerical_cols = ['year_hct','age_at_hct','donor_age']

ordinal_encoder = OrdinalEncoder()
onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
numerical_imputer = KNNImputer(n_neighbors=5)

combined_data = pd.concat([train, test], keys=['train', 'test'])

for feature in ordinal_numeric_cols:
    if feature in combined_data.columns:
        combined_data[feature] = combined_data[feature].fillna(-1)

for col in binary_cols:
    combined_data[col] = combined_data[col].fillna('unknown')
    combined_data[col] = combined_data[col].map({'No': 0, 'Yes': 1,'Not done': 2,'unknown': 2, 
                                'Positive': 1, 'Negative': 0})


for feature in ordinal_cols:
    if feature in combined_data.columns:
        combined_data[feature] = combined_data[feature].fillna('unknown')
        combined_data[feature] = ordinal_encoder.fit_transform(combined_data[[feature]])

'''
if nominal_cols:
    combined_data[nominal_cols] = combined_data[nominal_cols].fillna('unknown')
    combined_encoded = onehot_encoder.fit_transform(combined_data[nominal_cols])
    combined_encoded_df = pd.DataFrame(
        combined_encoded, 
        columns=onehot_encoder.get_feature_names_out(nominal_cols),
        index=combined_data.index
    )
    combined_data = combined_data.drop(columns=nominal_cols)
    combined_data = pd.concat([combined_data, combined_encoded_df], axis=1)
'''
combined_data[nominal_cols] = combined_data[nominal_cols].fillna('Unknown')
label_encoders = {}
for col in nominal_cols:
    label_encoder = LabelEncoder()
    combined_data[col] = label_encoder.fit_transform(combined_data[col])
    label_encoders[col] = label_encoder

combined_data[numerical_cols] = numerical_imputer.fit_transform(combined_data[numerical_cols])
combined_data['year_hct'] = 2024 - combined_data['year_hct']

#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#combined_data[numerical_cols] = scaler.fit_transform(combined_data[numerical_cols])
test = combined_data.xs('test')
train = combined_data.xs('train')



ID                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
dri_score                 [N/A - non-malignant indication, Intermediate,...
psych_disturb                                      [No, Not done, Yes, nan]
cyto_score                [nan, Intermediate, Poor, Other, Favorable, TB...
diabetes                                           [No, Yes, Not done, nan]
hla_match_c_high                                       [nan, 2.0, 1.0, 0.0]
hla_high_res_8                     [nan, 8.0, 7.0, 5.0, 6.0, 4.0, 3.0, 2.0]
tbi_status                [No TBI, TBI +- Other, >cGy, TBI + Cy +- Other...
arrhythmia                                         [No, Yes, nan, Not done]
hla_low_res_6                                [6.0, 4.0, 3.0, nan, 5.0, 2.0]
graft_type                                  [Bone marrow, Peripheral blood]
vent_hist                                                    [No, Yes, nan]
renal_issue                                        [No, nan, Yes, Not done]
pulm_severe 

In [5]:

import xgboost as xgb
from sklearn.model_selection import train_test_split
from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(train, duration_col='efs_time', event_col='efs')
train['risk_score'] = cph.predict_partial_hazard(train)

X = train.drop(columns=['efs_time', 'efs', 'risk_score'])
y = train['risk_score']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)


X_test = test.drop(columns=['efs_time', 'efs'])
test_predictions = xgb_model.predict(X_test)

submission = pd.DataFrame({
    'ID': test['ID'],
    'prediction': test_predictions
})


submission.to_csv('submission.csv', index=False)


print(submission.head())



/tmp/ipykernel_17/4145581175.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['risk_score'] = cph.predict_partial_hazard(train)


      ID  prediction
0  28800    0.447409
1  28801    1.968368
2  28802    0.410851
